In [5]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from extract_feature import *

In [2]:
dataset = {}
dataset['paper'] = pd.read_csv('dataRev2/Paper.csv')
dataset['author'] = pd.read_csv('dataRev2/Author.csv')
dataset['conference'] = pd.read_csv('dataRev2/Conference.csv')
dataset['journal'] = pd.read_csv('dataRev2/Journal.csv')
dataset['paper_author'] = pd.read_csv('dataRev2/PaperAuthor.csv')

In [23]:
def generate_feature_list(author_paper_pairs, ap_to_feature_list):
    result_list = []

    temp_dict = {} # { (author, paper) => [f1, f2 ...] }
    for ap_pair in author_paper_pairs:
        temp_dict[ap_pair] = []

    for i in range(len(ap_to_feature_list)):
        feature_dict = ap_to_feature_list[i]
        for ap_pair in author_paper_pairs:
            feature = feature_dict[ap_pair]
            temp_dict[ap_pair].append(feature)

    for key in temp_dict.keys():
        result_list.append(key + tuple( temp_dict[key] ))

    return result_list

def kamil_new_f1(dataset, author_paper_pairs):
    feature_dict = {}

    #aid_to_count = dataset['paper_author']['AuthorId'].value_counts()
    print(author_paper_pairs[0])
    for ap_pair in author_paper_pairs:
        feature_dict[ap_pair] = int(ap_pair[0]) + int(ap_pair[1])

    return feature_dict

def get_features(dataset, targetset):
    author_paper_pairs = parse_targetset(targetset)

    # Keep the format of f# (dictionary): { (a1, p1): feature_value1, (a2, p2): feature_value2 ... }
    # Add your features here and add them to feature_list!
    harry_f1 = get_author_publishes_how_many_paper_in_PaperAuthor(dataset, author_paper_pairs)
    harry_f2 = get_paper_has_how_many_author_in_PaperAuthor(dataset, author_paper_pairs)
    
    #harry_list = [harry_f1, harry_f2]
    harry_list = []
    
    
    kamil_f1 = kamil_new_f1(dataset, author_paper_pairs)
    kamil_list = [kamil_f1]
    
    thao_list = []
    feature_list = harry_list + kamil_list + thao_list

    result_list = generate_feature_list(author_paper_pairs, feature_list)
    return result_list

In [24]:
trainset = pd.read_csv('dataRev2/Train.csv')
train_confirmed = trainset[['AuthorId', 'ConfirmedPaperIds']].rename(columns = {'ConfirmedPaperIds':'PaperIds'})
print("Getting features for deleted papers from the database")
features_conf = get_features(dataset, train_confirmed)

Getting features for deleted papers from the database
(1539933, 1359549)


In [10]:
features_conf[:5]

[(1539933, 1359549, 7, 190),
 (1455231, 467172, 6, 267),
 (1215636, 1791266, 4, 446),
 (205278, 1737961, 4, 34),
 (433821, 1901940, 3, 262)]

In [ ]:
paper_author.head()

In [ ]:
pa_only = paper_author[['PaperId','AuthorId']]

In [ ]:
def find_relation(author_id,paper_id):
    relation_list = pd.DataFrame()
    #find papers related to the author
    related_papers = pa_only[pa_only.AuthorId == author_id]['PaperId'].unique()
    print("# related papers:",related_papers.shape)
    
    #find authors related to the paper
    related_authors = paper_author[paper_author['PaperId'] == paper_id]['AuthorId'].unique()
    print("# related authors:",related_authors.shape)
    
    #remove target paper and target author from lists
    related_papers = np.delete(related_papers,np.argwhere(related_papers == paper_id))
    related_authors = np.delete(related_authors, np.argwhere(related_authors == author_id))
    
    print("# related papers:",related_papers.shape)
    print("# related authors:",related_authors.shape)
    cnt = 0
    for author in related_authors:
        for paper in related_papers:
            single_relation = paper_author[(paper_author['PaperId'] == paper) & (paper_author['AuthorId'] == author)]
            if(single_relation.size > 0):
                #display(single_relation)
                relation_list = relation_list.append(single_relation)
                break
        """
        cnt += 1
        if cnt> 55:
            break
        """
    print(relation_list.shape)
    print(relation_list['AuthorId'].shape)
    
    return relation_list

#target author, target paper
d_list = [{"author" : "826", "confirmed" : "25733 47276 77012"},
          {"author" : "1118","confirmed" : "49963 93433 341015 415282"}
         ]

for d in d_list:
    author,confirmed = d["author"],d["confirmed"]
    for x in confirmed.split():
        tmp = find_relation(int(author),int(x))
        display(tmp)
        l.append(tmp)

In [ ]:
%timeit find_relation(826,25733)